In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTENC 
from sklearn .preprocessing import OneHotEncoder
from sklearn .preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time

In [2]:
data = pd.read_csv("Titanic.csv",sep=";")

In [3]:
print(data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [4]:
data = data.drop(columns=["Name","Ticket"])

In [5]:
data["Cabin"] = data["Cabin"].fillna(0)
data = data.dropna()

In [6]:
data = data.astype({"Cabin": str})

In [7]:
cible = data["Survived"]
data = data.drop(columns = "Survived" )

In [8]:
nb_samples = {
    0 : 1500,
    1 : 1500 
}

In [9]:

sm = SMOTENC(random_state=33,sampling_strategy=nb_samples,categorical_features=[3,5,8])

In [10]:
#encoder = OneHotEncoder()
#data = encoder.fit_transform(data.values.reshape(-1,1))

In [11]:
encoder = OrdinalEncoder()
data["Sex"] = encoder.fit_transform(data["Sex"].values.reshape(-1,1))
data["Cabin"] = encoder.fit_transform(data["Cabin"].values.reshape(-1,1))
data["Embarked"] = encoder.fit_transform(data["Embarked"].values.reshape(-1,1))


# Précision avant ingestion de nouvelles données

In [12]:
X_train, X_test, y_train, y_test = train_test_split( data.values, cible, test_size=0.3, random_state=4)

model_foret = RandomForestClassifier(random_state = 33 )
start_time = time.time()
model_foret.fit(X_train, y_train)
print("Temp d'entrainement: %s seconds " % (time.time() - start_time))


print("score test : ",model_foret.score(X_test, y_test))



Temp d'entrainement: 0.10671377182006836 seconds 
score test :  0.8271028037383178


In [13]:
data = sm.fit_resample(data,cible)

C:\Users\philg\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:300: UserWarning: After over-sampling, the number of samples (1500) in class 0 will be larger than the number of samples in the majority class (class #0 -> 424)
  f"After over-sampling, the number of samples ({n_samples})"
C:\Users\philg\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:300: UserWarning: After over-sampling, the number of samples (1500) in class 1 will be larger than the number of samples in the majority class (class #0 -> 424)
  f"After over-sampling, the number of samples ({n_samples})"


In [14]:
X = data[0]
y = data[1]
print(y)
print(X)

0       0
1       1
2       1
3       1
4       0
       ..
2995    1
2996    1
2997    1
2998    1
2999    1
Name: Survived, Length: 3000, dtype: int64
      PassengerId  Pclass       Sex   Age  SibSp  Parch       Fare  \
0               1       3  1.000000  22.0      1      0   7.250000   
1               2       1  0.000000  38.0      1      0  71.283300   
2               3       3  0.000000  26.0      0      0   7.925000   
3               4       1  0.000000  35.0      1      0  53.100000   
4               5       3  1.000000  35.0      0      0   8.050000   
...           ...     ...       ...   ...    ...    ...        ...   
2995          700       1  0.389126  31.0      0      0  30.426969   
2996          841       1  0.507218  16.0      0      1  19.614095   
2997          360       1  0.562426  24.0      0      0  36.870321   
2998          285       1  0.000000  30.0      0      0  79.424763   
2999          534       1  0.000000  16.0      0      0  99.882794   

      

# Précision après ingestion de nouvelles données

In [15]:
X_train, X_test, y_train, y_test = train_test_split( X.values, y, test_size=0.3, random_state=4)

start_time = time.time()
model_foret.fit(X_train, y_train)
print("Temp d'entrainement: %s seconds " % (time.time() - start_time))
y_pred = model_foret.predict(X_test)
print("score test : ",model_foret.score(X_test, y_test))



Temp d'entrainement: 0.23935794830322266 seconds 
score test :  0.8811111111111111
